In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import time
from itertools import islice
import tqdm
load_dotenv()

API_KEY = os.environ.get('SHIV_LASTFM_API_KEY','f35ca57df799d71fa75bcaeb13322fb6')
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

#https://www.last.fm/api/accounts

# No specific rate limits listed on the website

#### Table 1 - LasFM Tags / Genres

In [2]:
tracks_per_tag = 100
max_tags = 10


params = {
    "method": "tag.getTopTags",
    "api_key": API_KEY,
    "format": "json",
}
top_tags_r = requests.get(BASE_URL, params=params)
top_tags_data = top_tags_r.json()
tags_df = pd.DataFrame(top_tags_data['toptags']['tag'][:max_tags])
tags_df.head()

tags_df.to_csv('data/lastfm/tags_df.csv', index=False)
tags_df

,name,count,reach
0,rock,4052852,401853
1,electronic,2478813,260982
2,seen live,2183861,82507
3,alternative,2125391,266536
4,pop,2063259,232773
5,indie,2059424,259559
6,female vocalists,1630754,169029
7,metal,1297266,158486
8,alternative rock,1219438,169658
9,jazz,1189231,149487


### Table 2 - Tag Ranks

In [19]:
top_tags_data

{'toptags': {'@attr': {'offset': 0, 'num_res': 50, 'total': 2859101},
  'tag': [{'name': 'rock', 'count': 4052832, 'reach': 401850},
   {'name': 'electronic', 'count': 2478779, 'reach': 260978},
   {'name': 'seen live', 'count': 2183840, 'reach': 82507},
   {'name': 'alternative', 'count': 2125379, 'reach': 266533},
   {'name': 'pop', 'count': 2063224, 'reach': 232772},
   {'name': 'indie', 'count': 2059407, 'reach': 259559},
   {'name': 'female vocalists', 'count': 1630723, 'reach': 169029},
   {'name': 'metal', 'count': 1297241, 'reach': 158484},
   {'name': 'alternative rock', 'count': 1219424, 'reach': 169658},
   {'name': 'jazz', 'count': 1189219, 'reach': 149486},
   {'name': 'classic rock', 'count': 1147216, 'reach': 137051},
   {'name': 'ambient', 'count': 1114857, 'reach': 149272},
   {'name': 'experimental', 'count': 1102768, 'reach': 144480},
   {'name': 'folk', 'count': 951090, 'reach': 150826},
   {'name': 'indie rock', 'count': 919670, 'reach': 136765},
   {'name': 'punk'

In [26]:
tag_names = tags_df['name'].tolist()
all_tracks = []

for tag in tag_names[:1]:   
    tag_tracks = []
    page = 1

    while len(tag_tracks) < tracks_per_tag: 
        params = {
            "method": "tag.getTopTracks",
            "tag": tag,
            "api_key": API_KEY,
            "format": "json",
            "limit": 100,
            "page": page
        }
        response = requests.get(BASE_URL, params=params,headers= {"User-Agent": "JHU-Project/1.0 (spalit2@jh.edu)"})
        tracks_json = response.json()
        
        for track in tracks_json['tracks']['track']:
            track['tag'] = tag
            track['rank'] = int(track['@attr']['rank'])
            track['artist_name'] = track['artist']['name']
            track['artist_mbid'] = track['artist']['mbid']
        
        tag_tracks.extend(tracks_json['tracks']['track'])
        page += 1
        tracks_df = pd.DataFrame(tag_tracks)[['name','mbid','artist_name','artist_mbid','tag','rank']].rename({'mbid':'track_mbid','name':'track_name'},axis=1)
    
    break
        
    # for track in tag_tracks:
    #     track['tag'] = tag 
    
    # all_tracks.extend(tag_tracks)
    # print('Tag: ', tag, 'completed',':',len(tag_tracks))

In [27]:
page

2

In [28]:
tracks_df

,track_name,track_mbid,artist_name,artist_mbid,tag,rank
0,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,1
1,Mr. Brightside,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,rock,2
2,Creep,d11fcceb-dfc5-4d19-b45d-f4e8f6d3eaa6,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,rock,3
3,Come as You Are,e05035a3-14ac-4f88-a160-0a144530004e,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,4
4,Feel Good Inc.,5660558a-bfa7-416f-99d9-a34ca0a34515,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,rock,5
...,...,...,...,...,...,...
95,Song 2,b39b7227-c271-4ab8-912b-cb1fe2fc0e5e,Blur,ba853904-ae25-4ebb-89d6-c44cfbd71bd2,rock,96
96,Dream On,dec2c739-0853-4283-a85d-f5330e86f62b,Aerosmith,3d2b98e5-556f-4451-a3ff-c50ea18d57cb,rock,97
97,How to Save a Life,5c5cae19-b26c-4542-907a-9c15b51ab6a6,The Fray,2ddd167f-5b8a-4372-b350-6ad50493bac0,rock,98
98,Starlight,60e94685-0481-4d3d-bd84-11c389d9b2a5,Muse,fd857293-5ab8-40de-b29e-55a69d4e4d0f,rock,99


In [40]:
print(f"Getting tracks for tag: {tag}")
params = {
    "method": "tag.getTopTracks",
    "tag": tag,
    "api_key": API_KEY,
    "format": "json",
    "limit": 1000,
}

# call lastfm api
tracks_r = requests.get(BASE_URL, params=params)
tracks_json = tracks_r.json()

Getting tracks for tag: rock


In [44]:
len(tracks_json['tracks']['track'])

50

In [27]:
len(tracks_json['tracks']['track'])

50

In [18]:
tracks_df.groupby(['tag','track_name','artist_name'])

In [9]:
pd.DataFrame(all_tracks)

,name,duration,mbid,url,streamable,artist,image,@attr,tag,rank,artist_name,artist_mbid
0,Smells Like Teen Spirit,301,0ebe2d92-a11d-4b2b-9922-806383074ed7,https://www.last.fm/music/Nirvana/_/Smells+Lik...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Nirvana', 'mbid': '9282c8b4-ca0b-4c6...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '1'},rock,1,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6
1,Mr. Brightside,224,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,https://www.last.fm/music/The+Killers/_/Mr.+Br...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'The Killers', 'mbid': '95e1ead9-4d31...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '2'},rock,2,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b
2,Creep,239,d11fcceb-dfc5-4d19-b45d-f4e8f6d3eaa6,https://www.last.fm/music/Radiohead/_/Creep,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Radiohead', 'mbid': 'a74b1b7f-71a5-4...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '3'},rock,3,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711
3,Come as You Are,208,e05035a3-14ac-4f88-a160-0a144530004e,https://www.last.fm/music/Nirvana/_/Come+as+Yo...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Nirvana', 'mbid': '9282c8b4-ca0b-4c6...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '4'},rock,4,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6
4,Feel Good Inc.,221,5660558a-bfa7-416f-99d9-a34ca0a34515,https://www.last.fm/music/Gorillaz/_/Feel+Good...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Gorillaz', 'mbid': 'e21857d5-3256-45...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '5'},rock,5,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Summertime,695,a8661daf-63c6-4ef1-a746-ce162abb11ef,https://www.last.fm/music/John+Coltrane/_/Summ...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'John Coltrane', 'mbid': 'b625448e-bf...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '996'},jazz,996,John Coltrane,b625448e-bf4a-41c3-a421-72ad46cdb831
9996,Why Can't We Live Together,327,f8d77c6f-40dd-48cd-91ea-d06dd9e3e9f1,https://www.last.fm/music/Sade/_/Why+Can%27t+W...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Sade', 'mbid': '67930b3e-e00b-469f-8...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '997'},jazz,997,Sade,67930b3e-e00b-469f-8c74-fd69f20522ec
9997,Calibre,0,,https://www.last.fm/music/L%C3%B4+Borges/_/Cal...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Lô Borges', 'mbid': '5bddb0fa-66f7-4...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '998'},jazz,998,Lô Borges,5bddb0fa-66f7-47c8-bf00-afecab627d59
9998,As the Stars Fall,0,81e4a6c6-bc5f-4b62-8ab8-406d5730bec3,https://www.last.fm/music/The+Cinematic+Orches...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'The Cinematic Orchestra', 'mbid': '7...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '999'},jazz,999,The Cinematic Orchestra,7c158ea8-c0aa-410e-bdc1-20bba9759577


In [38]:
tag_rank_df = pd.DataFrame(all_tracks)[['name','mbid','artist_name','artist_mbid','tag','rank']].rename({'mbid':'track_mbid','name':'track_name'},axis=1)
tag_rank_df.to_csv('data/lastfm/tag_rank_df.csv')
tag_rank_df


,track_name,track_mbid,artist_name,artist_mbid,tag,rank
0,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,1
1,Mr. Brightside,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,rock,2
2,Creep,d11fcceb-dfc5-4d19-b45d-f4e8f6d3eaa6,Radiohead,a74b1b7f-71a5-4011-9441-d0b5e4122711,rock,3
3,Come as You Are,e05035a3-14ac-4f88-a160-0a144530004e,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,4
4,Feel Good Inc.,5660558a-bfa7-416f-99d9-a34ca0a34515,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,rock,5
...,...,...,...,...,...,...
9995,Summertime,a8661daf-63c6-4ef1-a746-ce162abb11ef,John Coltrane,b625448e-bf4a-41c3-a421-72ad46cdb831,jazz,996
9996,Why Can't We Live Together,f8d77c6f-40dd-48cd-91ea-d06dd9e3e9f1,Sade,67930b3e-e00b-469f-8c74-fd69f20522ec,jazz,997
9997,As the Stars Fall,81e4a6c6-bc5f-4b62-8ab8-406d5730bec3,The Cinematic Orchestra,7c158ea8-c0aa-410e-bdc1-20bba9759577,jazz,998
9998,Temptation,e015a013-5083-4701-9331-4afbdf9f9ceb,Diana Krall,67d2cb7a-9ddb-4a7f-82bf-5a2d1a038e98,jazz,999


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/lastfm/tag_rank_df.csv')

In [7]:
df[df['rank'] == 1]


,Unnamed: 0,track_name,track_mbid,artist_name,artist_mbid,tag,rank
0,0,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,rock,1
1000,1000,Feel Good Inc.,5660558a-bfa7-416f-99d9-a34ca0a34515,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,electronic,1
2000,2000,espresso,NaN,Sabrina Carpenter,1882fe91-cdd9-49c9-9956-8e06a3810bd4,seen live,1
3000,3000,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,alternative,1
4000,4000,Feel Good Inc.,5660558a-bfa7-416f-99d9-a34ca0a34515,Gorillaz,e21857d5-3256-4547-afb3-4b6ded592596,pop,1
5000,5000,Mr. Brightside,37d516ab-d61f-4bcb-9316-7a0b3eb845a8,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,indie,1
6000,6000,Poker Face,c50edf48-ac2b-442f-b24f-d2acf775441c,Lady Gaga,650e7db6-b795-4eb5-a702-5ea2fc46c848,female vocalists,1
7000,7000,In the End,9d70086c-5d7a-4e7f-b1ed-c53c4b11310f,Linkin Park,f59c5520-5f46-4d2c-b2c4-822eabf53419,metal,1
8000,8000,Smells Like Teen Spirit,0ebe2d92-a11d-4b2b-9922-806383074ed7,Nirvana,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,alternative rock,1
9000,9000,Back to Black,b0adc7a8-0a90-4e09-8839-a66f315234a2,Amy Winehouse,dfe9a7c4-8cf2-47f4-9dcb-d233c2b86ec3,jazz,1


### Table 3 - Track Info

In [32]:

mbids = tag_rank_df['track_mbid'].unique()
tracks = []
for mbid in tqdm.tqdm(mbids):
    try:
        params = {
            "method": "track.getInfo",
            "api_key": API_KEY,
            "mbid": mbid,
            "format": "json"
        }
        response = requests.get(BASE_URL, params=params)
        tracks.append(response.json())
    except:
        print('Error: ', mbid)

    
tracks_df = pd.DataFrame([t['track'] for t in tracks])


  0%|          | 0/4508 [00:00<?, ?it/s]

0ebe2d92-a11d-4b2b-9922-806383074ed7


In [39]:
tracks_df.to_csv('data/lastfm/tracks_df.csv', index=False)
tracks_df

,name,mbid,url,duration,streamable,listeners,playcount,artist,album,toptags,wiki
0,Knocked Up,6956c27e-3c15-494f-bff9-af2241f2506c,https://www.last.fm/music/Kings+of+Leon/_/Knoc...,430000,"{'#text': '0', 'fulltrack': '0'}",491594,3039772,"{'name': 'Kings of Leon', 'mbid': '6ffb8ea9-23...","{'artist': 'Kings of Leon', 'title': 'Because ...","{'tag': [{'name': 'rock', 'url': 'https://www....","{'published': '13 Jun 2008, 20:12', 'summary':..."
1,Need You Tonight,aa12dc94-77b1-4e9b-990f-b2bae959f5e0,https://www.last.fm/music/INXS/_/Need+You+Tonight,189000,"{'#text': '0', 'fulltrack': '0'}",802799,4589591,"{'name': 'INXS', 'mbid': '481bf5f9-2e7c-4c44-b...","{'artist': 'INXS', 'title': 'Kick', 'mbid': '1...","{'tag': [{'name': '80s', 'url': 'https://www.l...","{'published': '31 Jan 2009, 17:14', 'summary':..."
2,Fitter Happier,5838f978-0822-4e28-874f-e1511324ec3a,https://www.last.fm/music/Radiohead/_/Fitter+H...,117000,"{'#text': '0', 'fulltrack': '0'}",1070145,6165992,"{'name': 'Radiohead', 'mbid': 'a74b1b7f-71a5-4...","{'artist': 'Radiohead', 'title': 'OK Computer'...","{'tag': [{'name': 'alternative', 'url': 'https...","{'published': '31 Mar 2009, 19:12', 'summary':..."
3,Mercy,e528371a-0544-4328-a51e-8a2427671fcc,https://www.last.fm/music/Duffy/_/Mercy,220000,"{'#text': '0', 'fulltrack': '0'}",980004,5628145,"{'name': 'Duffy', 'mbid': 'da4f7583-bb31-49d2-...","{'artist': 'Duffy', 'title': 'Deluxe EP', 'mbi...","{'tag': [{'name': 'soul', 'url': 'https://www....","{'published': '08 Nov 2008, 11:10', 'summary':..."
4,Blue Ridge Mountains,310fa155-f188-46bb-b39a-f5ab8ac048df,https://www.last.fm/music/Fleet+Foxes/_/Blue+R...,200000,"{'#text': '0', 'fulltrack': '0'}",701071,5366356,"{'name': 'Fleet Foxes', 'mbid': 'fa97dd36-1b82...","{'artist': 'Fleet Foxes', 'title': 'Fleet Foxe...","{'tag': [{'name': 'folk', 'url': 'https://www....",NaN
...,...,...,...,...,...,...,...,...,...,...,...
4501,This Fire,da22fa3b-ee21-4326-adda-cfb2dabaaecf,https://www.last.fm/music/Franz+Ferdinand/_/Th...,255000,"{'#text': '0', 'fulltrack': '0'}",853912,4660231,"{'name': 'Franz Ferdinand', 'mbid': 'aa7a2827-...","{'artist': 'Franz Ferdinand', 'title': 'Franz ...","{'tag': [{'name': 'indie', 'url': 'https://www...","{'published': '29 Jan 2009, 13:52', 'summary':..."
4502,Forgotten,241e711d-e836-433c-b4bd-b2340f199f7b,https://www.last.fm/music/Linkin+Park/_/Forgotten,194000,"{'#text': '0', 'fulltrack': '0'}",759399,5715870,"{'name': 'Linkin Park', 'mbid': 'f59c5520-5f46...","{'artist': 'Linkin Park', 'title': 'Hybrid The...","{'tag': [{'name': 'Nu Metal', 'url': 'https://...","{'published': '26 Jun 2024, 10:18', 'summary':..."
4503,Shimmy,668b2928-dbc8-401a-9b2d-6fe546dcd441,https://www.last.fm/music/System+of+a+Down/_/S...,111000,"{'#text': '0', 'fulltrack': '0'}",765557,4783874,"{'name': 'System of a Down', 'mbid': 'cc0b7089...","{'artist': 'System of a Down', 'title': 'Great...","{'tag': [{'name': 'alternative metal', 'url': ...",NaN
4504,Mermaid,42640092-938d-49b7-a5ff-2a54f3a33175,https://www.last.fm/music/Sade/_/Mermaid,264000,"{'#text': '0', 'fulltrack': '0'}",154552,756496,"{'name': 'Sade', 'mbid': '67930b3e-e00b-469f-8...","{'artist': 'Sade', 'title': 'Love Deluxe', 'mb...","{'tag': [{'name': 'soul', 'url': 'https://www....","{'published': '02 Aug 2024, 17:30', 'summary':..."


In [33]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="localhost",
    database="group6",
    user="group6",
    password="group6",
    port="5432"
)

cur = conn.cursor()



In [35]:
MAX_TAGS = 25
TRACKS_PER_TAG = 100
HEADERS = {"User-Agent": "JHU-Project/1.0 (spalit2@jh.edu)"}
api_key = 'f35ca57df799d71fa75bcaeb13322fb6'
base_url = 'http://ws.audioscrobbler.com/2.0/'
# get top tags from db
with conn:
    with conn.cursor() as cur:
       
        cur.execute("SELECT tag_name FROM lastfm_top_tags")
        top_tags = [tag[0] for tag in cur.fetchall()]

        all_tracks_initial = []
        track_data = []

        # get tracks for each tag
        for tag in top_tags:
            print(f"Getting tracks for tag: {tag}")
            tag_tracks = []
            page = 1
            while len(tag_tracks) < TRACKS_PER_TAG:
                time.sleep(0.2)
                params = {
                    "method": "tag.getTopTracks",
                    "tag": tag,
                    "api_key": api_key,
                    "format": "json",
                    "limit": TRACKS_PER_TAG,
                    "page": page
                }

                response = requests.get(base_url, params=params, headers=HEADERS)
                tracks = response.get("tracks", {}).get("track", [])
                if not tracks:
                    break

                for track in tracks:
                    track["tag"] = tag

                tag_tracks.extend(tracks)
                page += 1
            tag_tracks = tag_tracks[:TRACKS_PER_TAG]
            all_tracks_initial.extend(tag_tracks)

Getting tracks for tag: rock


AttributeError: 'Response' object has no attribute 'get'

In [36]:
import uuid

In [44]:
from datetime import datetime, timedelta
import os
import zipfile
import subprocess
import pandas as pd
import uuid




# Retrieves a specific file from kaggle via API
def get_kag_file():

    # User needs to set up account on kaggle and generate API key
    os.environ['KAGGLE_USERNAME'] = 'zmbarrett'
    os.environ['KAGGLE_KEY'] = 'eca1fde2463f087bbeea1ee3b57d4400'

    # Retrieving a specific dataset
    dataset_name = 'elizabethearhart/billboard-hot-1001958-2024'
    zip_file_name = f"{dataset_name.split('/')[-1]}.zip"
    
    # Data is to go into data subdirectory
    data_dir = os.path.join(os.getcwd(), 'data')
    os.makedirs(data_dir, exist_ok=True)

    # Download the dataset by running the kaggle command
    subprocess.run([
        'kaggle', 'datasets', 'download', 
        '-d', dataset_name, 
        '--force', '-p', data_dir])

    # Unzip to data subdirectory
    zip_path = os.path.join(data_dir, zip_file_name)
    try:

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir) 
            extracted_files = zip_ref.namelist()

        if extracted_files:
            extracted_file_name = extracted_files[0]
            print(f"Extracted Kaggle file: {extracted_file_name} to data directory")

    except PermissionError:
        print("Error extracting ZIP file: Permission denied")
        print("Close Kaggle data csv if you already have it open in another program")
        return None

    # Extract first file
    extracted_file_name = extracted_files[0] if extracted_files else None
    print(f"Extracted Kaggle file: {extracted_file_name} to data directory")

    # Keep extracted file, delete zip
    os.remove(zip_path)
    print(f"The ZIP file {zip_file_name} has been deleted\n")

    return extracted_file_name


# Processes kaggle file in data directory and returns a dataframe
def process_kag_file(file_name):
    print(f"\nProcessing Kaggle file: {file_name} ...\n"
          "File contains Bilboard Hot 100 chart data from 1958 to 2024\n"
          "It contains - chart_week, title, performer, current_week, last_week, peak_pos, wks_on_chart\n"
          "Duplicate song title, performer pairs will be removed\n")

    data_dir = os.path.join(os.getcwd(), 'data')
    os.makedirs(data_dir, exist_ok=True)
    kaggle_file_path = os.path.join(data_dir, file_name)

    # Read kaggle file from data directory
    df = pd.read_csv(kaggle_file_path, encoding='utf-8')

    # Strip and lowercase string cols
    df['title'] = df['title'].str.strip().str.lower()
    df['performer'] = df['performer'].str.strip().str.lower()

    # Force chart_wk to date for sorting
    df['chart_week'] = pd.to_datetime(df['chart_week'], errors='coerce')

    # Replace empty strings with None
    df.replace('', None, inplace=True)

    df_kaggle = df.copy()

    # Replace some strings to help with matching to Spotify data
    df_kaggle['performer'] = df_kaggle['performer'].str.replace(r' \(featuring ([^)]*)\)', r',\1', regex=True)
    df_kaggle['performer'] = df_kaggle['performer'].str.replace(r' \(feat. ([^)]*)\)', r',\1', regex=True)
    df_kaggle['performer'] = df_kaggle['performer'].str.replace('featuring', ',')
    df_kaggle['performer'] = df_kaggle['performer'].str.replace('feat.', ',')
    df_kaggle['performer'] = df_kaggle['performer'].str.replace('&', 'and')
    df_kaggle['performer'] = df_kaggle['performer'].str.replace(r' ,', r',', regex=True)
    df_kaggle['title'] = df_kaggle['title'].str.replace(r' \(featuring.*$', '', regex=True)

    # Force int columns to numeric, force any NaNs to None
    int_columns = ['current_week', 'last_week', 'peak_pos', 'wks_on_chart']
    for col in int_columns:
        df_kaggle[col] = pd.to_numeric(df_kaggle[col], errors='coerce')
    df_kaggle[int_columns] = df_kaggle[int_columns].where(pd.notnull(df_kaggle[int_columns]), None)

    # Force int columns to Int64 (needed for one column that was being read as float)
    # Replace 0 with None
    for col in int_columns:
        df_kaggle[col] = df_kaggle[col].astype('Int64')
    for col in int_columns:
        df_kaggle[col] = df_kaggle[col].replace(0, None)


    print(f"Number of Kaggle dataset rows before removing duplicates: {len(df_kaggle)}")

    # First, group by performer and title to find first and last chart dates
    date_groups = df_kaggle.groupby(['performer', 'title']).agg({
        'chart_week': ['min', 'max'],  # Get both first and last dates
        'current_week': 'last',  # Keep values from the most recent entry
        'peak_pos': 'min',      # Always keep lowest (best) peak position
        'wks_on_chart': 'max'   # Keep highest weeks on chart value
    })
    
    # Flatten MultiIndex columns
    date_groups.columns = ['wk_first_charted', 'wk_last_charted', 'last_chart_pos', 
                           'peak_chart_pos', 'total_wks_on_chart']
    date_groups = date_groups.reset_index()
    
    # Convert back to the original dataframe format with the additional column
    df_kaggle = date_groups.copy()

    print(f"Number of Kaggle dataset rows after removing duplicates: {len(df_kaggle)}\n")

    # Rename columns
    df_kaggle.rename(columns={'performer': 'artists',
                           'title': 'song_name'}, inplace=True)
    
    # Reorder columns
    df_kaggle = df_kaggle[['artists', 'song_name', 'peak_chart_pos',
                           'last_chart_pos', 'total_wks_on_chart', 
                           'wk_first_charted', 'wk_last_charted']]


    # Write to CSV for record
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_hot100 = f"unique_hot100_{timestamp}.csv"
    kag_file_name = os.path.join(data_dir, unique_hot100)

    df_kaggle.to_csv(kag_file_name, index=False,
                        encoding='utf-8-sig')
    print(f"Kaggle dataset processed and saved to {unique_hot100}\n")

    count = (df_kaggle['peak_chart_pos'] == 1).sum()
    print(f"Processed Kaggle dataset contains {len(df_kaggle)} songs in total and " 
          f"{count} songs that peaked at #1")
    print(f"Dataframe has columns: {', '.join(df_kaggle.columns)}\n")


    # Return the processed dataframe
    return df_kaggle

In [45]:
kag_name = get_kag_file()
df_kaggle = process_kag_file(kag_name)

Dataset URL: https://www.kaggle.com/datasets/elizabethearhart/billboard-hot-1001958-2024
License(s): MIT

Extracted Kaggle file: hot-100-current.csv to data directory
Extracted Kaggle file: hot-100-current.csv to data directory
The ZIP file billboard-hot-1001958-2024.zip has been deleted


Processing Kaggle file: hot-100-current.csv ...
File contains Bilboard Hot 100 chart data from 1958 to 2024
It contains - chart_week, title, performer, current_week, last_week, peak_pos, wks_on_chart
Duplicate song title, performer pairs will be removed



100%|██████████| 3.20M/3.20M [00:00<00:00, 1.85GB/s]


Number of Kaggle dataset rows before removing duplicates: 343600
Number of Kaggle dataset rows after removing duplicates: 31337

Kaggle dataset processed and saved to unique_hot100_20250420_163748.csv

Processed Kaggle dataset contains 31337 songs in total and 1173 songs that peaked at #1
Dataframe has columns: artists, song_name, peak_chart_pos, last_chart_pos, total_wks_on_chart, wk_first_charted, wk_last_charted



In [65]:
df_kaggle['artists'].str.split(',').str[0]

0                         "groove" holmes
1                         "groove" holmes
2                  "little" jimmy dickens
3                         "pookie" hudson
4                     "weird al" yankovic
                       ...               
31332    ¥$: kanye west and ty dolla $ign
31333    ¥$: kanye west and ty dolla $ign
31334            ¥$: ye and ty dolla $ign
31335            ¥$: ye and ty dolla $ign
31336            ¥$: ye and ty dolla $ign
Name: artists, Length: 31337, dtype: object

In [74]:

import base64
from urllib.parse import urlencode

client_id = 'd2d5bf12b79f4947be8568a2a4105a93'
client_secret = '61ec80164ca14449a0588e4e64330fda'

# Get a Spotify access token
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
headers = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded",
}
data = {"grant_type": "client_credentials"}

print("Requesting an access token from Spotify")
response = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
if response.status_code == 200:
    token = response.json().get("access_token")
    token_type = response.json().get("token_type")
    expires_in = response.json().get("expires_in")
    print(f"Received a Spotify {token_type} access token that expires in {expires_in} seconds\n")
else:
    print("Error:", response.status_code, response.text)


headers = {"Authorization": f"Bearer {token}"}

# Harcoding good playlists for our purposes
# Need to do dynamically, Spotify seems to rotate playlist IDs
playlist_queries = [
        "Billboard Hot 100: All #1 hit songs 1958-2024",
        "Top 1000 greatest songs of all time",
        "100 Greatest Rock Songs",
        "Top 100 hip hop hits of all time",
        "100 Greatest Pop Songs",
        "Top 100 Most Popular Electronic Songs Of All Time",
        "Top 100 Alternative Rock Songs",
        "Top 100 Jazz Songs",
        "The 100 Greatest Heavy Metal Songs of All Time",
        "100 Best Folk Songs"
    ]

# Dictionary to store playlist information
playlist_info = {}

# Search for each playlist and retrieve its ID
for query in playlist_queries:
    time.sleep(1)
    print(f"Searching for playlist: \"{query}\"...")
    
    # Set up search parameters
    search_url = "https://api.spotify.com/v1/search"
    params = {
        "q": query,
        "type": "playlist",
        "limit": 1
    }
    
    # Make the search request
    response = requests.get(f"{search_url}?{urlencode(params)}", headers=headers)
    print(response.json()['playlists']['items'])

Requesting an access token from Spotify
Received a Spotify Bearer access token that expires in 3600 seconds

Searching for playlist: "Billboard Hot 100: All #1 hit songs 1958-2024"...
[{'collaborative': False, 'description': 'A playlist with all number one hit songs on the Billboard Hot 100 list.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/7c2c13pKxvFDSV4WSyydyg'}, 'href': 'https://api.spotify.com/v1/playlists/7c2c13pKxvFDSV4WSyydyg', 'id': '7c2c13pKxvFDSV4WSyydyg', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da842e19fe95567ba0f68d46b9b8', 'width': None}], 'name': 'Billboard Hot 100: All #1 hit songs 1958-2024', 'owner': {'display_name': 'Juan Ceñal', 'external_urls': {'spotify': 'https://open.spotify.com/user/ordago13'}, 'href': 'https://api.spotify.com/v1/users/ordago13', 'id': 'ordago13', 'type': 'user', 'uri': 'spotify:user:ordago13'}, 'primary_color': None, 'public': True, 'snapshot_id': 'AAACdANyOK3RFPwS0lKyjIIkE

In [70]:
response.json()

{'playlists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=Top%201000%20greatest%20songs%20of%20all%20time&type=playlist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=Top%201000%20greatest%20songs%20of%20all%20time&type=playlist',
  'offset': 0,
  'previous': None,
  'total': 806,
  'items': [{'collaborative': False,
    'description': 'Listen to the currently 1000 most popular songs from 1950 to 1999. You can listen to the greatest songs of all time in this playlist. This playlist is listed most popular to least popular song. Weekly updated (Last: 02-09-25). Cover: The Police. Support my band by listening to Revenge!',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/7epfU4cGVZ1xHWQbQJl7km'},
    'href': 'https://api.spotify.com/v1/playlists/7epfU4cGVZ1xHWQbQJl7km',
    'id': '7epfU4cGVZ1xHWQbQJl7km',
    'images': [{'height': None,
      'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da84b5a